<a href="https://colab.research.google.com/github/KamilianovAA/HiraganaRecognition/blob/master/hiragana_recognition_KAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras import backend as K
import numpy as np
import tensorflow_hub as hub

In [2]:
batch_size = 128
num_classes = 49
epochs = 7

# input image dimensions
img_rows, img_cols = 28, 28
path = './sample_data/'

def load(f):
    return np.load(f)['arr_0']

# Load the data
x_train = load(path + 'k49-train-imgs.npz')
x_test = load(path + 'k49-test-imgs.npz')
y_train = load(path + 'k49-train-labels.npz')
y_test = load(path + 'k49-test-labels.npz')

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('{} train samples, {} test samples'.format(len(x_train), len(x_test)))

232365 train samples, 38547 test samples


In [3]:
# Convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [15]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [16]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 32)        320       
                                                                 
 conv2d_3 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 64)       0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 12, 12, 64)        0         
                                                                 
 flatten_1 (Flatten)         (None, 9216)              0         
                                                                 
 dense_3 (Dense)             (None, 128)               1179776   
                                                      

In [17]:
model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/7
1816/1816 [==============================] - 621s 342ms/step - loss: 0.9402 - accuracy: 0.7503 - val_loss: 0.6479 - val_accuracy: 0.8272
Epoch 2/7
1816/1816 [==============================] - 623s 343ms/step - loss: 0.5269 - accuracy: 0.8551 - val_loss: 0.5019 - val_accuracy: 0.8663
Epoch 3/7
1816/1816 [==============================] - 612s 337ms/step - loss: 0.4263 - accuracy: 0.8812 - val_loss: 0.4417 - val_accuracy: 0.8840
Epoch 4/7
1816/1816 [==============================] - 631s 348ms/step - loss: 0.3688 - accuracy: 0.8957 - val_loss: 0.4023 - val_accuracy: 0.8940
Epoch 5/7
1816/1816 [==============================] - 623s 343ms/step - loss: 0.3330 - accuracy: 0.9045 - val_loss: 0.3822 - val_accuracy: 0.9006
Epoch 6/7
1816/1816 [==============================] - 621s 342ms/step - loss: 0.3053 - accuracy: 0.9118 - val_loss: 0.3757 - val_accuracy: 0.9035
Epoch 7/7
1816/1816 [==============================] - 622s 342ms/step - loss: 0.2813 - accuracy: 0.9181 - val_loss: 0

In [ ]:
model = keras.models.load_model('hiragana_model.h5')

In [19]:
train_score = model.evaluate(x_train, y_train, verbose=0)
test_score = model.evaluate(x_test, y_test, verbose=0)
print('Train loss:', train_score[0])
print('Train accuracy:', train_score[1])
print('Test loss:', test_score[0])
print('Test accuracy:', test_score[1])

Train loss: 0.0852271243929863
Train accuracy: 0.9766358733177185
Test loss: 0.3620363771915436
Test accuracy: 0.9084753394126892


In [18]:
model.save('hiragana_model.h5')